# Income Analysis per User
In this section, we calculate the total income generated by each user for the company. We will create an interactive plot using Plotly, which allows us to click on a point representing the income generated by a user and see details like the user ID and the date of the request.

### Monthly Income Analysis
Next, we want to analyze the income on a monthly basis. We will extract the year and month from the created_at column, calculate the total monthly income per user, and then compute the average monthly income per user. This will allow us to visualize how the average income per user varies over time.

### Extract Year and Month
We will extract the year and month from the created_at column to facilitate monthly analysis.

### Calculate Monthly Income and User Count
We will calculate the total monthly income per user and the number of unique users per month. This allows us to determine the average monthly income per user.

### Calculate Average Monthly Income per User
Using the calculated monthly income and user count, we will compute the average monthly income per user.

### Second Interactive Plot
We create a line plot to visualize the average monthly income per user over time. This interactive plot allows us to observe trends and patterns in the average income.

### Formatted Table of Average Monthly Income
Finally, we create a formatted table to display the average monthly income per user. This table provides a clear and concise summary of the average income for each month.

This script provides a detailed analysis of the income generated by each user, including interactive visualizations and a formatted table to facilitate easy understanding and exploration of the data.

In [39]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

# Cargar los datos
fees_data = pd.read_csv('../cleaned_dataset/modified_fees_data.csv')
cash_requests_data = pd.read_csv('../cleaned_dataset/modified_cash_requests_data.csv')

# Convertir 'created_at' a datetime
fees_data['created_at'] = pd.to_datetime(fees_data['created_at'])
cash_requests_data['created_at'] = pd.to_datetime(cash_requests_data['created_at'])

# Calcular el ingreso total generado por cada usuario
user_income = fees_data.groupby('cash_request_id')['total_amount'].sum().reset_index()
user_income = user_income.merge(cash_requests_data[['id', 'user_id', 'created_at']], left_on='cash_request_id', right_on='id', how='left')

# Primer gráfico interactivo usando Plotly
fig1 = px.scatter(user_income, x='total_amount', y='user_id', size='total_amount', color='total_amount',
                 hover_data=['user_id', 'created_at'],
                 labels={'total_amount': 'Income Generated', 'user_id': 'User ID'},
                 title='Income Generated by User')
fig1.update_layout(transition_duration=500)
fig1.show()

# Extraer el año y el mes
user_income['year_month'] = user_income['created_at'].dt.to_period('M').astype(str)

# Calcular el ingreso total mensual por usuario
monthly_income = user_income.groupby(['user_id', 'year_month'])['total_amount'].sum().reset_index()

# Calcular el número de usuarios por mes
user_count_per_month = user_income.groupby('year_month')['user_id'].nunique().reset_index()
user_count_per_month.columns = ['year_month', 'user_count']

# Calcular el ingreso promedio mensual por usuario
monthly_avg_income = monthly_income.groupby('year_month')['total_amount'].sum().reset_index()
monthly_avg_income = monthly_avg_income.merge(user_count_per_month, on='year_month')
monthly_avg_income['avg_income_per_user'] = monthly_avg_income['total_amount'] / monthly_avg_income['user_count']

# Segundo gráfico interactivo usando Plotly
fig2 = px.line(monthly_avg_income, x='year_month', y='avg_income_per_user', 
               title='Average Monthly Income per User', 
               hover_data={'avg_income_per_user': True, 'user_count': True})
fig2.update_layout(transition_duration=500, xaxis_title='Month', yaxis_title='Average Income per User')
fig2.show()

# Crear una tabla bonita y formateada con el promedio mensual y el número de usuarios
fig3 = go.Figure(data=[go.Table(
    header=dict(values=['Year-Month', 'Average Income per User', 'Number of Users'],
                fill_color='paleturquoise',
                align='left'),
    cells=dict(values=[monthly_avg_income['year_month'], monthly_avg_income['avg_income_per_user'], monthly_avg_income['user_count']],
               fill_color='lavender',
               align='left'))
])

fig3.update_layout(title='Average Monthly Income per User Table')
fig3.show()


# Incident and Income Analysis Over the Last 7 Months
In this section, we analyze the incident rates and income over the last 7 months. We will create a line plot to compare incident rates and income, and bar plots to show the proportion of different statuses and transfer types.

### Data Preparation
We start by converting the created_at columns to datetime format and filtering the data for the last 7 months.

### Ignore Specific Warnings
We ignore specific warnings about converting to period format.

### Convert Columns to Datetime
We ensure that the created_at columns are in datetime format for both datasets.

### Filter Data for the Last 7 Months
We filter the data to include only the last 7 months.

### Calculate Incidents and Income
We calculate the number of incidents and the total income for each month within the filtered data.

### Align Periods
We ensure that all periods are aligned to cover the entire range of the last 7 months, filling any missing periods with zeros.

### Prepare Data for Plotting
We prepare a DataFrame containing the month, incident rates, and income for visualization.

### Visualization of Incidents and Income
We create a line plot using Plotly to compare incident rates and income over the last 7 months.

### Line Plot for Incident Rates and Income
This plot shows the trend of incident rates and income, with each line representing a different metric.

### Calculate Proportions for Legend
We calculate the proportion of each status and transfer type to display these values in the legends of the bar plots.

### Status Proportions
We calculate and format the proportions of different statuses for display.

### Transfer Type Proportions
We calculate and format the proportions of different transfer types for display.

### Bar Plots for Proportions
We create bar plots using Plotly to show the proportion of different statuses and transfer types over the last 7 months.

### Bar Plot for Status Proportions
This plot displays the count and proportion of each status.

### Bar Plot for Transfer Type Proportions
This plot displays the count and proportion of each transfer type.

### Display Proportions in Table Format
We create tables to display the count and proportion of different statuses and transfer types.

### Status Proportion Table
We create a table showing the count and proportion of each status.

### Transfer Type Proportion Table
We create a table showing the count and proportion of each transfer type.

This analysis provides insights into the trends in incident rates and income, as well as the distribution of different statuses and transfer types over the last 7 months. The interactive visualizations and formatted tables make it easy to explore and understand the data.

In [35]:
import warnings

# Ignorar advertencias específicas sobre la conversión de la información de zona horaria
warnings.filterwarnings("ignore", message="Converting to PeriodArray/Index representation will drop timezone information.")

# Asegurar que las columnas 'created_at' están en formato datetime
fees_data['created_at'] = pd.to_datetime(fees_data['created_at'])
cash_requests_data['created_at'] = pd.to_datetime(cash_requests_data['created_at'])

# Filtrar los datos para los últimos 7 meses
recent_months = fees_data['created_at'].dt.to_period('M').sort_values().unique()[-7:]
filtered_fees = fees_data[fees_data['created_at'].dt.to_period('M').isin(recent_months)]
filtered_cash_requests = cash_requests_data[cash_requests_data['created_at'].dt.to_period('M').isin(recent_months)]

# Calcular incidentes e ingresos
monthly_incidents = filtered_fees[filtered_fees['type'] == 'incident'].groupby(filtered_fees['created_at'].dt.to_period('M')).size()
monthly_income = filtered_fees.groupby(filtered_fees['created_at'].dt.to_period('M'))['total_amount'].sum()

# Asegurar que todos los períodos estén alineados
all_periods = pd.period_range(start=recent_months.min(), end=recent_months.max(), freq='M')
monthly_incidents = monthly_incidents.reindex(all_periods, fill_value=0)
monthly_income = monthly_income.reindex(all_periods, fill_value=0)

# Preparar datos para el gráfico
data = pd.DataFrame({
    'Month': all_periods.astype(str),
    'Incident Rates': monthly_incidents.values,
    'Income': monthly_income.values
})

# Visualización con Plotly para comparar incidentes e ingresos
fig1 = px.line(data, x='Month', y=['Incident Rates', 'Income'],
              labels={'value': 'Quantity', 'variable': 'Type'},
              title='Comparison of Incident Rates and Income Over the Last 7 Months')
fig1.update_traces(mode='lines+markers')
fig1.update_layout(
    legend=dict(
        title='Type',
        itemsizing='constant',
        font=dict(size=12)
    )
)
fig1.show()

# Calcular proporción y formatear para la leyenda
status_counts = filtered_fees['status'].value_counts()
status_proportion = status_counts / status_counts.sum()
status_labels = [f"{status} ({count}, {percent:.2%})" for status, count, percent in zip(status_counts.index, status_counts, status_proportion)]

transfer_counts = filtered_cash_requests['transfer_type'].value_counts()
transfer_proportion = transfer_counts / transfer_counts.sum()
transfer_labels = [f"{transfer_type} ({count}, {percent:.2%})" for transfer_type, count, percent in zip(transfer_counts.index, transfer_counts, transfer_proportion)]

# Gráfico de barras de Plotly para proporciones
fig2 = px.bar(x=status_counts.index, y=status_counts.values, labels={'x': 'Status', 'y': 'Proportion'},
             title='Proportion of Status Over the Last 7 Months')
fig2.update_traces(text=[f"{percent:.2%}" for percent in status_proportion], textposition='outside')
fig2.update_layout(
    xaxis={'categoryorder':'total descending'},
    showlegend=False
)
fig2.show()

fig3 = px.bar(x=transfer_counts.index, y=transfer_counts.values, labels={'x': 'Transfer Type', 'y': 'Proportion'},
             title='Proportion of Transfer Type Over the Last 7 Months')
fig3.update_traces(text=[f"{percent:.2%}" for percent in transfer_proportion], textposition='outside')
fig3.update_layout(
    xaxis={'categoryorder':'total descending'},
    showlegend=False
)
fig3.show()

# Mostrar proporciones con nombres personalizados en la leyenda
status_proportion_table = pd.DataFrame({
    'Status': status_counts.index,
    'Count': status_counts.values,
    'Proportion': status_proportion.values
})

transfer_type_proportion_table = pd.DataFrame({
    'Transfer Type': transfer_counts.index,
    'Count': transfer_counts.values,
    'Proportion': transfer_proportion.values
})


# Detailed Analysis of Incidents by Fee Status and Transfer Type
In this section, we perform a detailed analysis of incidents by fee status and transfer type. We will create a comprehensive figure that includes line plots for monthly incident counts and bar plots for incident counts by fee status and transfer type.

### Data Preparation
We start by converting the created_at columns to datetime format and filtering the data to include only incidents.

### Convert Columns to Datetime
We ensure that the created_at columns are in datetime format for both datasets.

### Filter Data for Incidents
We filter the fees data to include only records marked as incidents.

### Merge Data for Detailed Analysis
We merge the incident data with the cash requests data to obtain more details, ensuring both status columns are retained with distinct names.

### Add Year-Month Column
We add a year_month column to both datasets to facilitate monthly grouping.

### Calculate Monthly Incidents and Transactions
We calculate the number of incidents and total transactions for each month.

### Merge Incident and Transaction Data
We merge the monthly incident counts with the monthly transaction counts to calculate the incident percentage for each month.

### Count Incidents by Fee Status and Transfer Type
We calculate the number of incidents by fee status and by transfer type, and compute their proportions.

### Create Line and Bar Plots
We create a comprehensive figure using subplots that includes line plots for the monthly incident counts and bar plots for incident counts by fee status and transfer type.

### Line Plots for Monthly Incidents
We create line plots to show the total incidents, and incidents broken down by transfer type and fee status, over time.

### Bar Plots for Incident Counts by Fee Status and Transfer Type
We create bar plots to show the count and proportion of incidents by fee status and transfer type.

### Combine Plots into a Single Figure
We use plotly.subplots to combine the line and bar plots into a single figure.

### Update Layout
We adjust the layout of the figure to ensure it is large enough and the font sizes are appropriate for readability.

This analysis provides a detailed view of incident trends over time and the distribution of incidents by fee status and transfer type. The interactive visualizations facilitate easy exploration and understanding of the data.

In [38]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Convertir columnas 'created_at' a datetime
fees_data['created_at'] = pd.to_datetime(fees_data['created_at'])
cash_requests_data['created_at'] = pd.to_datetime(cash_requests_data['created_at'])

# Filtrar por incidentes en los datos de tarifas
incident_data = fees_data[fees_data['type'] == 'incident']

# Unir datos de incidentes con datos de solicitudes para obtener más detalles
# Mantener ambas columnas 'status' con alias diferentes
incident_data = incident_data.merge(
    cash_requests_data[['id', 'transfer_type', 'status']],
    left_on='cash_request_id', 
    right_on='id', 
    how='left',
    suffixes=('_fee', '_cash_request')
)

# Agregar columna year_month para agrupar por mes
incident_data['year_month'] = incident_data['created_at'].dt.to_period('M')
cash_requests_data['year_month'] = cash_requests_data['created_at'].dt.to_period('M')

# Contar incidentes y transacciones totales por mes
monthly_incidents = incident_data.groupby('year_month').size().reset_index(name='Incident Count')
monthly_transactions = cash_requests_data.groupby('year_month').size().reset_index(name='Transaction Count')

# Unir incidentes con transacciones para calcular el porcentaje
monthly_data = monthly_incidents.merge(monthly_transactions, on='year_month')
monthly_data['Incident Percentage'] = monthly_data['Incident Count'] / monthly_data['Transaction Count'] * 100
monthly_data['year_month'] = monthly_data['year_month'].astype(str)

# Contar incidentes por status de tarifas
incident_status_counts = incident_data['status_fee'].value_counts().reset_index()
incident_status_counts.columns = ['Status', 'Incident Count']
incident_status_counts['Proportion'] = incident_status_counts['Incident Count'] / incident_status_counts['Incident Count'].sum()

# Contar incidentes por tipo de transferencia
incident_transfer_type_counts = incident_data['transfer_type'].value_counts().reset_index()
incident_transfer_type_counts.columns = ['Transfer Type', 'Incident Count']
incident_transfer_type_counts['Proportion'] = incident_transfer_type_counts['Incident Count'] / incident_transfer_type_counts['Incident Count'].sum()

# Crear gráficos de líneas adicionales para cada transfer_type y status
# Incidentes por tipo de transferencia
transfer_type_data = incident_data.groupby(['year_month', 'transfer_type']).size().reset_index(name='Incident Count')
transfer_type_data = transfer_type_data.merge(monthly_transactions, on='year_month')
transfer_type_data['Incident Percentage'] = transfer_type_data['Incident Count'] / transfer_type_data['Transaction Count'] * 100
transfer_type_data['year_month'] = transfer_type_data['year_month'].astype(str)

# Incidentes por status de tarifas
status_data = incident_data.groupby(['year_month', 'status_fee']).size().reset_index(name='Incident Count')
status_data = status_data.merge(monthly_transactions, on='year_month')
status_data['Incident Percentage'] = status_data['Incident Count'] / status_data['Transaction Count'] * 100
status_data['year_month'] = status_data['year_month'].astype(str)

# Crear una figura con subplots
fig = make_subplots(
    rows=2, cols=2, 
    specs=[[{"colspan": 2}, None], [{"type": "bar"}, {"type": "bar"}]],
    subplot_titles=("Monthly Incident Count Over Time", "Incident Count by Fee Status", "Incident Count by Transfer Type"),
    vertical_spacing=0.2,
    horizontal_spacing=0.1
)

# Agregar el gráfico de líneas
fig.add_trace(
    go.Scatter(x=monthly_data['year_month'], y=monthly_data['Incident Count'],
               mode='lines+markers', name='Total Incidents',
               text=monthly_data['Incident Percentage'].map(lambda x: f"{x:.2f}%"),
               hovertemplate='%{y} incidents (%{text})<extra></extra>'),
    row=1, col=1
)

# Agregar el gráfico de líneas para incidentes por tipo de transferencia
for transfer_type in transfer_type_data['transfer_type'].unique():
    data = transfer_type_data[transfer_type_data['transfer_type'] == transfer_type]
    fig.add_trace(
        go.Scatter(x=data['year_month'], y=data['Incident Count'],
                   mode='lines+markers', name=f'Transfer Type: {transfer_type}',
                   text=data['Incident Percentage'].map(lambda x: f"{x:.2f}%"),
                   hovertemplate='%{y} incidents (%{text})<extra></extra>'),
        row=1, col=1
    )

# Agregar el gráfico de líneas para incidentes por status
for status in status_data['status_fee'].unique():
    data = status_data[status_data['status_fee'] == status]
    fig.add_trace(
        go.Scatter(x=data['year_month'], y=data['Incident Count'],
                   mode='lines+markers', name=f'Status: {status}',
                   text=data['Incident Percentage'].map(lambda x: f"{x:.2f}%"),
                   hovertemplate='%{y} incidents (%{text})<extra></extra>'),
        row=1, col=1
    )

# Agregar gráfico de barras: Incidentes por Status de Tarifas
fig.add_trace(
    go.Bar(x=incident_status_counts['Status'], y=incident_status_counts['Incident Count'],
           text=incident_status_counts.apply(lambda row: f"{row['Incident Count']} ({row['Proportion']:.2%})", axis=1),
           textposition='outside', name='Status'),
    row=2, col=1
)

# Agregar gráfico de barras: Incidentes por Tipo de Transferencia
fig.add_trace(
    go.Bar(x=incident_transfer_type_counts['Transfer Type'], y=incident_transfer_type_counts['Incident Count'],
           text=incident_transfer_type_counts.apply(lambda row: f"{row['Incident Count']} ({row['Proportion']:.2%})", axis=1),
           textposition='outside', name='Transfer Type'),
    row=2, col=2
)

# Actualizar el layout
fig.update_layout(
    height=1000,  # Aumentar la altura de la figura
    showlegend=True,
    title_text="Analysis of Incidents by Fee Status and Transfer Type",
    title_font_size=20,
    margin=dict(l=50, r=50, t=100, b=50),
    font=dict(size=14)  # Aumentar el tamaño de la fuente
)

# Ajustar el tamaño de las subtramas
fig.update_xaxes(tickangle=45, tickfont=dict(size=12))
fig.update_yaxes(tickfont=dict(size=12))

# Mostrar la figura
fig.show()
